# 3.3 Exercise: Working with OpenEO on Copernicus Data
Now that we familiarized the concepts of this OER, it's time for us to get to the practical part and take a look how that works together. First of all please run the first code snippet and set your preferred output directory, where the resulting .tif files can be saved, in the input.

In [2]:
output_dir = input("Please enter your preferred output directory as an whole path: ")

## 3.3.1 Installing the needed software
First of all we need to import the needed python libraries. We need to import the openeo library for getting our data first. After that we also need the numpy, matplotlib, rasterio, skimage and os libraries for plotting our result in the end. 
If you're interested to learn more about the openeo python library, you can find the detailed documentation here: https://openeo.org/documentation/1.0/python/.

In [3]:
import openeo
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from skimage import exposure
import os



## 3.3.2 Sign in with your Account
We want to use the Sentinel 2 data of the Copernicus program. OpenEO gives us the opportunity to work with the Copernicus Data Space Ecosystem which can provide us with the datasets we need. To get access to the data we need to connect with the Copernicus Data Space Ecosystem and sign in with our account. This is possbile with the connect() function of the openeo library. Here you also can connect to other services like Google Earth Engine. On this site you get the information which url you need for your choosen service: https://hub.openeo.org/. But for this tutorial we will use the Copernicus Data Space Ecosystem.    
We need to sing in with our account to get access to the data. This is possible with the authenticate_*() functions. We will use the authenticate_oidc() function. It's the OpenID Connect authentification. With this function it's possbile to sign in from the browser into your account. Therefore you need to click on the link in the output.

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")

connection.authenticate_oidc()

## 3.3.3 Creating arguments for the Sentinel 2 data
Before we can access our data, we have to create some arguments. We only need a small area around Lorup and also set a time period of one day to only get the data of one Sentinel 2 shot. There is also the option of getting all the data in a specific time period. But in our use case of the calculation of the NDVI that isn't necessary.

In [5]:
# Define the date for a specific day before the flood
date_before = ["2021-06-12", "2021-06-14"]

# Define the date for a specific day after the flood
date_after = ["2021-07-20", "2021-07-22"]

# Define the bounding box for the flood region Ahrweiler 
flood_region_bbox = {
    "west": 7.055496548917159,
    "south": 50.52018836230043,
    "east": 7.117575039388905,
    "north": 50.546364682516895
} 

## 3.3.4 load Sentinel 2 data with datacube
Now we can load our wanted data. This is possbile through Datacubes. Datacubes are a central concept in openEO to represent EO data (if you want to know more about it, follow this link: https://openeo.org/documentation/1.0/datacubes.html. To get a datacube object of the Sentinel 1 data we need another function of the OpenEO library load_connection().
For the calculation of the NDVI we only need the B04 (red) and B08 (near-infrared) bands of the Sentinel 2 data, therefore we specify that as an additional parameter in the function.

In [6]:
# Load Sentinel-2 data including the SCL band for cloud masking
s2_data_before = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=flood_region_bbox,
    temporal_extent=date_before,
    bands=["B04", "B08"]
)

s2_data_after = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=flood_region_bbox,
    temporal_extent=date_after,
    bands=["B04", "B08"]
)

## 3.3.5 Calculating the NDVI
The NDVI (Normalized Difference Vegetation Index) is calculated using the near-infrared (B08) and red (B04) bands. This is a common index used to measure vegetation health. 
Luckily for us there already exists a function for that from the OpenEO API. OpenEO has many predefined functions like this, called processes, to perform calculations or mainpulate your data. These processes are also documented and can be found here: https://openeo.org/documentation/1.0/processes.html.

In [7]:
# NDVI calculation for both time periods
ndvi_before = s2_data_before.ndvi(nir="B08", red="B04")
ndvi_after = s2_data_after.ndvi(nir="B08", red="B04")

## 3.3.6 Saving the NDVI Result as a GeoTIFF
We save the resulting NDVI image as a GeoTIFF file with a filename that includes the date range.

In [8]:
# Reduce dimension to average the results over the time period
ndvi_mean_before = ndvi_before.reduce_dimension(dimension="t", reducer="mean")
ndvi_mean_after = ndvi_after.reduce_dimension(dimension="t", reducer="mean")

# Save the pre- and post-flood NDVI results
result_before = ndvi_mean_before.save_result(format="GTiff")
result_after = ndvi_mean_after.save_result(format="GTiff")

## 3.3.7 Creating and Starting a Batch Job
We create a batch job to process the data on the OpenEO backend. The job is then started and we wait for it to complete.

In [ ]:
# Create jobs and download results for "Before" NDVI
job_before = result_before.create_job()
job_before.start_and_wait()
if job_before.status() == "finished":
    job_before.get_results().download_files(output_dir)
    print("NDVI before flood downloaded successfully.")
else:
    print("Error in downloading NDVI before flood!")

# Rename downloaded files based on job completion order
downloaded_files = os.listdir(output_dir)
for filename in downloaded_files:
    if filename.endswith(".tif"):
        file_path = os.path.join(output_dir, filename)
        if "Before" not in filename and "After" not in filename:
            if not os.path.exists(os.path.join(output_dir, "NDVI_Before_Flood.tif")):
                print(f"Renaming {filename} to NDVI_Before_Flood.tif")
                os.rename(file_path, os.path.join(output_dir, "NDVI_Before_Flood.tif"))
            elif not os.path.exists(os.path.join(output_dir, "NDVI_After_Flood.tif")):
                print(f"Renaming {filename} to NDVI_After_Flood.tif")
                os.rename(file_path, os.path.join(output_dir, "NDVI_After_Flood.tif"))


# Create jobs and download results for "After" NDVI
job_after = result_after.create_job()
job_after.start_and_wait()
if job_after.status() == "finished":
    job_after.get_results().download_files(output_dir)
    print("NDVI after flood downloaded successfully.")
else:
    print("Error in downloading NDVI after flood!")

# Rename downloaded files based on job completion order
downloaded_files = os.listdir(output_dir)
for filename in downloaded_files:
    if filename.endswith(".tif"):
        file_path = os.path.join(output_dir, filename)
        if "Before" not in filename and "After" not in filename:
            if not os.path.exists(os.path.join(output_dir, "NDVI_Before_Flood.tif")):
                print(f"Renaming {filename} to NDVI_Before_Flood.tif")
                os.rename(file_path, os.path.join(output_dir, "NDVI_Before_Flood.tif"))
            elif not os.path.exists(os.path.join(output_dir, "NDVI_After_Flood.tif")):
                print(f"Renaming {filename} to NDVI_After_Flood.tif")
                os.rename(file_path, os.path.join(output_dir, "NDVI_After_Flood.tif"))


## 3.3.9 Show resulting .tif file
As our last step we finally visualize the NDVI data using a color map that highlights vegetation. 

In [ ]:
# Load and plot the pre- and post-flood NDVI images
for filename in ["NDVI_Before_Flood.tif", "NDVI_After_Flood.tif"]:
    file_path = os.path.join(output_dir, filename)
    if os.path.exists(file_path):
        with rasterio.open(file_path) as src:
            data = src.read(1)
            data = np.clip(data, -1, 1)

        # Plot the NDVI
        plt.figure(figsize=(10, 10))
        plt.title(f'NDVI Image - {filename}')
        plt.imshow(data, cmap='RdYlGn', vmin=-1, vmax=1)
        plt.colorbar()
        plt.show()
    else:
        print(f"File {filename} not found!")

# Function to enhance contrast using contrast stretching
def enhance_contrast(ndvi_data):
    # Stretching contrast
    p2, p98 = np.percentile(ndvi_data, (2, 98))  # Clip the 2nd and 98th percentile values
    ndvi_enhanced = exposure.rescale_intensity(ndvi_data, in_range=(p2, p98))
    return ndvi_enhanced

# Load and plot enhanced NDVI images
def plot_enhanced_ndvi(filename, output_dir):
    file_path = os.path.join(output_dir, filename)
    with rasterio.open(file_path) as src:
        ndvi_data = src.read(1)
        ndvi_data = np.clip(ndvi_data, -1, 1)  # Clip NDVI values to [-1, 1]
        
        # Enhance contrast
        ndvi_enhanced = enhance_contrast(ndvi_data)
        
        # Plot the enhanced NDVI
        plt.figure(figsize=(10, 10))
        plt.title(f'Enhanced NDVI Image - {filename}')
        plt.imshow(ndvi_enhanced, cmap='RdYlGn', vmin=-1, vmax=1)
        plt.colorbar()
        plt.show()

In [ ]:
from skimage import exposure
import matplotlib.pyplot as plt
import rasterio

# Function to enhance contrast using contrast stretching
def enhance_contrast(ndvi_data):
    # Stretching contrast
    p2, p98 = np.percentile(ndvi_data, (2, 98))  # Clip the 2nd and 98th percentile values
    ndvi_enhanced = exposure.rescale_intensity(ndvi_data, in_range=(p2, p98))
    return ndvi_enhanced

# Load and plot enhanced NDVI images
def plot_enhanced_ndvi(filename, output_dir):
    file_path = os.path.join(output_dir, filename)
    with rasterio.open(file_path) as src:
        ndvi_data = src.read(1)
        ndvi_data = np.clip(ndvi_data, -1, 1)  # Clip NDVI values to [-1, 1]
        
        # Enhance contrast
        ndvi_enhanced = enhance_contrast(ndvi_data)
        
        # Plot the enhanced NDVI
        plt.figure(figsize=(10, 10))
        plt.title(f'Enhanced NDVI Image - {filename}')
        plt.imshow(ndvi_enhanced, cmap='RdYlGn', vmin=-1, vmax=1)
        plt.colorbar()
        plt.show()

# Example of plotting the enhanced images
output_dir = "C:/Users/Tobias/Desktop/OER-CopernicusAndopenEO/"
plot_enhanced_ndvi("NDVI_Before_Flood.tif", output_dir)
plot_enhanced_ndvi("NDVI_After_Flood.tif", output_dir)

# Paths to the before and after NDVI images
before_path = os.path.join(output_dir, "NDVI_Before_Flood.tif")
after_path = os.path.join(output_dir, "NDVI_After_Flood.tif")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from skimage import exposure

# Function to calculate NDVI difference
def calculate_ndvi_difference(before_path, after_path):
    with rasterio.open(before_path) as before_src:
        ndvi_before = before_src.read(1)
    
    with rasterio.open(after_path) as after_src:
        ndvi_after = after_src.read(1)
    
    # Clip NDVI values between -1 and 1
    ndvi_before = np.clip(ndvi_before, -1, 1)
    ndvi_after = np.clip(ndvi_after, -1, 1)
    
    # Calculate the difference
    ndvi_diff = ndvi_after - ndvi_before
    
    return ndvi_diff

# Function to enhance contrast using percentiles (2nd and 98th)
def enhance_contrast(ndvi_diff):
    p2, p98 = np.percentile(ndvi_diff, (2, 98))  # Compute the 2nd and 98th percentiles
    ndvi_diff_enhanced = exposure.rescale_intensity(ndvi_diff, in_range=(p2, p98))
    return ndvi_diff_enhanced

# Function to plot the enhanced NDVI difference
def plot_enhanced_ndvi_difference(ndvi_diff_enhanced):
    plt.figure(figsize=(10, 10))
    plt.title('Enhanced NDVI Difference (After - Before)')
    plt.imshow(ndvi_diff_enhanced, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar(label='NDVI Difference')
    plt.show()

# Paths to the before and after NDVI images
before_path = os.path.join(output_dir, "NDVI_Before_Flood.tif")
after_path = os.path.join(output_dir, "NDVI_After_Flood.tif")

# Calculate the NDVI difference
ndvi_difference = calculate_ndvi_difference(before_path, after_path)

# Enhance the contrast using the 2nd and 98th percentiles
ndvi_difference_enhanced = enhance_contrast(ndvi_difference)

# Plot the enhanced NDVI difference
plot_enhanced_ndvi_difference(ndvi_difference_enhanced)
